In [1]:

from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,confusion_matrix,recall_score,precision_score,f1_score 
import matplotlib.pylab as plt
import numpy as np

In [2]:
ubdata_set=pd.read_csv(r"D:\portfolio projects\python\Naive biad loan approval\UniversalBank.csv")
ubdata_set1=ubdata_set.drop(labels=['ZIP Code','ID'] ,axis=1)
ubdata_set1.describe()
#ubdata_set1.head(5000)

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,45.338400,20.104600,73.774200,2.396400,1.937938,1.881000,56.498800,0.096000,0.104400,0.06040,0.596800,0.294000
std,11.463166,11.467954,46.033729,1.147663,1.747659,0.839869,101.713802,0.294621,0.305809,0.23825,0.490589,0.455637
min,23.000000,-3.000000,8.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,35.000000,10.000000,39.000000,1.000000,0.700000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,45.000000,20.000000,64.000000,2.000000,1.500000,2.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000
75%,55.000000,30.000000,98.000000,3.000000,2.500000,3.000000,101.000000,0.000000,0.000000,0.00000,1.000000,1.000000
max,67.000000,43.000000,224.000000,4.000000,10.000000,3.000000,635.000000,1.000000,1.000000,1.00000,1.000000,1.000000


In [3]:
# Searching for null values
ubdata_set1.isna().sum()

Age                   0
Experience            0
Income                0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal Loan         0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
dtype: int64

In [4]:
ubdata_set2=ubdata_set1[['Personal Loan','CreditCard','Online']].copy()
ubdata_set2


,Personal Loan,CreditCard,Online
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,1,0
...,...,...,...
4995,0,0,1
4996,0,0,1
4997,0,0,0
4998,0,0,1


### Splitting data into 60:40 as train and test sets

In [10]:
# spliting the data
train,test = train_test_split(ubdata_set2, test_size=0.4,random_state=12)

### Normalising the data

In [13]:
dfnormalize= preprocessing.normalize(train)
print(type(train))
print(train.columns)
train_norm=pd.DataFrame(train, columns=ubdata_set2.columns)
train_norm.describe()

<class 'pandas.core.frame.DataFrame'>
Index(['Personal Loan', 'CreditCard', 'Online'], dtype='object')


,Personal Loan,CreditCard,Online
count,3000.000000,3000.000000,3000.000000
mean,0.096667,0.299000,0.592667
std,0.295553,0.457896,0.491420
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000
75%,0.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000


In [12]:
pd.melt(train_norm, id_vars=['Personal Loan'], value_vars=['CreditCard', 'Online'])

,Personal Loan,variable,value
0,0,CreditCard,1
1,0,CreditCard,0
2,0,CreditCard,1
3,1,CreditCard,0
4,0,CreditCard,1
...,...,...,...
5995,0,Online,0
5996,0,Online,0
5997,0,Online,0
5998,0,Online,0


In [8]:
train_norm.pivot_table(index=['Personal Loan','CreditCard','Online'],aggfunc=len )

Personal Loan  CreditCard  Online
0              0           0          769
                           1         1131
               1           0          332
                           1          478
1              0           0           82
                           1          121
               1           0           39
                           1           48
dtype: int64

### classifying a customer who owns a bank credit card and is actively using online banking services. Looking at the pivot table the probability of 
### loan acceptance (Loan = 1) conditional on having a bank credit card (CC = 1) and being an active user of online banking services (Online = 1)]
### The probability of loan acceptence by customer of total records 526.out of 48 people got approved loan
### 48/(48+478)=0.0912

### Create two separate pivot tables for the training data. One will have Loan (rows) as a function of Online (columns)

In [13]:
train_norm.pivot_table(index=['Personal Loan','Online'],aggfunc=len )


CreditCard
Personal Loan Online            
0             0             1101
              1             1609
1             0              121
              1              169

### Compute the following quantities [P(A | B) means “the probability ofA given B”]:i. P(CC = 1 | Loan = 1)

In [30]:
train_norm.pivot_table(index=['Personal Loan','CreditCard'],aggfunc=len )

Online
Personal Loan CreditCard        
0             0             1900
              1              810
1             0              203
              1               87

### probability  P((Pl=1/Online=1)
### 169/(169+121)=0.582
### P(Persnol Loan = 0/Online = 1 )
### 1609/(1609+1101)=0.59

### Probability P((Personal Loan=1)/CC=1)
### 87/(87+203)=0.28
### probabilityP(Personal Loan = 0/CC = 1 ) 
### 810/(810+190)=0.29

In [17]:
train_norm.pivot_table(index=['Personal Loan','CreditCard','Online'],aggfunc=len )

Personal Loan  CreditCard  Online
0              0           0          769
                           1         1131
               1           0          332
                           1          478
1              0           0           82
                           1          121
               1           0           39
                           1           48
dtype: int64

### P(PL=1) here the total people accepted loan is 290 from the total 3000 persons.
### 290/(3000)=0.09666=9.6 % acceptence rate
### P[PL=0] same here the people who rejected the loan 
### 2710/(2710+290)=0.9033=90 % of rejection rate

###  NAIVE BAYES probability P(Loan = 1 | CC = 1, Online = 1)
### (0.28*0.587*0.09666)/((0.28*0.587*0.0966)+(0.29*0.59*0.9033))=0.0924

In [16]:
#Naive Bayes 
predictor = np.array(train_norm.iloc[:,[1,2]] )
target = np.array(train_norm['Personal Loan'])
X_train,X_test,y_train,y_test = train_test_split(predictor,target,test_size=0.4,random_state=12)

In [17]:
 #Initialize the object and training data
Gs = GaussianNB()
Gs.fit(X_train,y_train)

GaussianNB()

In [18]:
#predictions
Gs_y_predict = Gs.predict(X_test)
print(Gs_y_predict)

[0 0 0 ... 0 0 0]


In [567]:
# prediction accuracy
print(accuracy_score(y_test,Gs_y_predict))

0.9025


### The values are newar to each other to tell which one is more accurate. But based on the lecture, I would say pivot table B because Naive Bayes is that it assumes that all predictors are independent. It says that pivot table B is more accurate because it won't depend on the predictors to be independent, and this is where in pivot table B the relationship between the predictors stay.
### Here,I  can see that the outcome is the same .0924= 9.24% which is out of those 5000 customers only 9.24% accepted the loan.